# RedPanda Examples

In [1]:
import pandas, redpanda, sqlalchemy, sys
print("RedPanda version:", redpanda.__version__)
print("Python version:", sys.version)

RedPanda version: 0.2.1
Python version: 3.5.1 (default, May 30 2016, 16:01:04) 
[GCC 5.3.0]


## Opening a session

Open a `RedPanda` session to begin exploring data:

In [2]:
engine = redpanda.create_engine("sqlite://")
session = redpanda.Session(bind=engine)

Note that this is syntactic sugar for SQLAlchemy calls:

In [3]:
engine = sqlalchemy.create_engine("sqlite://")
sessionmaker = sqlalchemy.orm.sessionmaker(class_=redpanda.orm.Session,
                                           query_cls=redpanda.orm.Query)
session = sessionmaker(bind=engine)

## Creating An Example Model

Define your models using the declarative API. `RedPanda` defines one such model in the `example` module:

In [4]:
import sqlalchemy.ext.declarative
Base = sqlalchemy.ext.declarative.declarative_base()
class Widget(Base):
    id        = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    timestamp = sqlalchemy.Column(sqlalchemy.DateTime)
    name      = sqlalchemy.Column(sqlalchemy.String)
    kind      = sqlalchemy.Column(sqlalchemy.String)
    units     = sqlalchemy.Column(sqlalchemy.Integer)

    __read_sql__  = {"index_col": ["timestamp"], "parse_dates": ["timestamp"]}
    __tablename__ = "widgets"

Defining the class attribute `__read_sql__` will configure default arguments passed to `pandas.read_sql()`. 

In this case we are configuring the `widgets` table to use the `timestamp` column as an index and that we wish for `pandas` to parse this column as a `datetime` object.

In [5]:
from redpanda.example import create_widgets
create_widgets(session)

## Querying Data

Use the `frame()` method of a query to return the results as a `DataFrame`:

In [6]:
session.query(Widget).frame().head()

,id,name,kind,units
timestamp,,,,
2016-01-06 20:18:00,1,suit,bopper,85
2016-01-22 04:45:00,2,cause,fizzer,46
2016-01-22 05:26:00,3,recesses,buzzer,12
2016-01-30 02:53:00,4,valve,bopper,85
2016-02-05 01:12:00,5,forecast,fizzer,40


### Querying with Filters

Use SQLAlchemy filters to refine the query:

In [7]:
session.query(Widget).filter(Widget.timestamp.between("2016-11-01", "2016-11-30")).frame()

,id,name,kind,units
timestamp,,,,
2016-11-14 14:40:00,64,dependencies,bopper,90
2016-11-21 02:39:00,65,wish,bopper,67
2016-11-24 17:40:00,66,pairs,buzzer,91


### Overriding Defaults

Pass arguments to `pandas.read_sql()` into the `frame()` method:

In [8]:
session.query(Widget).frame(index_col="id").head()

,timestamp,name,kind,units
id,,,,
1,2016-01-06 20:18:00,suit,bopper,85
2,2016-01-22 04:45:00,cause,fizzer,46
3,2016-01-22 05:26:00,recesses,buzzer,12
4,2016-01-30 02:53:00,valve,bopper,85
5,2016-02-05 01:12:00,forecast,fizzer,40


## Writing DataFrame to Session

Use the `add_dataframe()` method of the `RedPanda` session to place `DataFrame` rows into the session as models:

In [9]:
from datetime import datetime
dfdata = {
    "kind": "newkind", 
    "name": "newname", 
    "units": 1000, 
    "timestamp": datetime.utcnow()}
addme = pandas.DataFrame(dfdata, index=[0])
addme

,kind,name,timestamp,units
0,newkind,newname,2016-07-06 14:43:44.590633,1000


In [10]:
session.add_dataframe(Widget, addme)
session.commit()
session.query(Widget).filter(Widget.kind=="newkind").frame()

,id,name,kind,units
timestamp,,,,
2016-07-06 14:43:44.590633,76,newname,newkind,1000
